In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import os
from random import shuffle
from tqdm import tqdm
import csv
import sys
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')
import keras
sys.stderr = stderr
from keras import *
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.utils import plot_model
from keras.models import load_model
import cv2 as cv
import Augmentor


image_colmn = 40
image_row  = 40


In [3]:


def imagify(origin_array, size):
    #function to turn a 1d vector into a square matrix
    #origin_aray -> any vector
    #size -> the size of the matrix to create
    new_array = np.zeros((size,size))
    for i in range (0,size):
        for j in range (0,size):
            new_array[i][j] = origin_array[i*size+j]
    return new_array

def de_imagify(img, size):
    #function to turn a square matrix into a vector
    #img -> square matrix
    #size -> the size of the square matrix
    new_array = np.zeros((size ** 2))
    for i in range(size):
        for j in range(size):
            new_array[i*size+j] = img[i][j]
    return np.asarray(new_array)

def crop_all_images(input_file_path, output_file_path):
    #function to take all the images from the input_file_path, and crop them to a uniform size
    #by default, all images are cropped and rescaled to 100,100
    #the resulting images are saved into the output_file_path
    #return: the size of the biggest cropped image, before rescaling
    all_img = np.load(input_file_path, encoding='latin1')

    #make an identical copy of the file, we will only modify the data of the images
    cropped_img = all_img.copy()
    #make a list to store the cropped images temporarily
    cropped_list = []

    #variables storing the size of the biggest image, used to resize all the samples
    max_width = 0
    max_height = 0
    for i in tqdm(range(all_img.shape[0])):
        #get the image in this row
        img = imagify(all_img[i][1],100)
        #make a copy that will remain unaltered
        img_cpy = img.copy()
        #blur the image
        img = cv.GaussianBlur(img,(3,3),0)
        # convert to grayscale
        imgray = np.uint8(img * 255) 
        #convert to binary image
        ret, thresh = cv.threshold(imgray, 20, 255, 0)
        #get the contours in the image
        im2, contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        #convert to rgb to have 3 channels
        im2 = cv.cvtColor(im2, cv.COLOR_GRAY2RGB)   
        #now get the biggest contour from the image
        maxArea = 0
        maxIndex = 0
        if len(contours) != 0:
            for i in range(len(contours)):
                if cv.contourArea(contours[i]) > maxArea:
                    maxArea = cv.contourArea(contours[i])
                    maxIndex = i
        #get the coordinates of the rectangle surrounding the shape
        a,b,c,d = cv.boundingRect(contours[maxIndex])
        #draw the rectangle
        #cv.rectangle(img,(a,b),(a+c,b+d),(255),1)
        #crop the original image
        crop = img_cpy[b:b+d, a:a+c]
        temp_max = np.max([c,d])
        crop = cv.resize(crop,(100,100))
        cropped_img[i][1] = de_imagify(crop,100)
        cropped_list.append(crop)
        if (c > max_width):
            max_width = c
        if (d > max_height):
            max_height = d

    #get the max size, i.e. biggest value between width and height
    max_size = np.max([max_height,max_width])
    for i in tqdm(range(cropped_img.shape[0])):
    #for i in tqdm(range(2)):
        #resize the array
        np.resize(cropped_img[i][1],(max_size ** 2))
        #cropped_img[i][1].resize(max_size)
        img = cropped_list[i]
        #crop the image to the max size, as a square
        crop = cv.resize(img,(max_size,max_size))
        #cropped_img[i][1] = de_imagify(crop,max_size)
        cropped_img[i][1] = de_imagify(crop,max_size)
        #crop = cv.resize(crop,(100,100))


    np.save(output_file_path, cropped_img)
    return max_size

def resize_all(input_file_path,output_file_path, current_size, size):
    #function to resize all the images in a file
    #takes the images in input_file_path and puts the resized ones in output_file_path
    #current_size -> the current size of the square matrices representing the images
    #size -> the wanted size of the square matrices
    all_img = np.load(input_file_path, encoding='latin1')
    print(all_img.shape[0])
    all_copy = all_img.copy()
    img_list = []
    for i in tqdm(range(all_img.shape[0])):
        img = imagify(all_img[i][1],current_size)
        #resize the array
        np.resize(all_img[i][1],(size ** 2))
        resized_img = cv.resize(img,(size,size))
        all_img[i][1] = de_imagify(resized_img,size)
    np.save(output_file_path,all_img)



In [4]:
def create_train_data():
    class_label_name=[]
    with open (train_label_filename,'r',) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            
            class_label_name.append(row)
    del class_label_name[0]
    class_id_list = []
    for i in range(len(class_name_list)):
        class_id_list.append(i)
        
    hashmap = dict(zip(class_name_list,class_id_list))
    class_label_vector=[]

    for r in tqdm(class_label_name):
        word = r[1]
        if word in hashmap:
            class_label_vector.append(hashmap[word])
    images = np.load(read_train_filename,encoding='latin1')
    train_data=[]
    for i in tqdm(range(len(images))):
            train_data.append([np.array((images[i][1]).reshape(1,image_colmn*image_row)),np.array(class_label_vector[i])])
    np.save('train_data.npy',train_data)
    return train_data


In [5]:
def elastic_transform(image, alpha, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    assert len(image.shape) == 2

    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), indexing='ij')
    indices = np.reshape(x + dx, (-1, 1)), np.reshape(y + dy, (-1, 1))

    return map_coordinates(image, indices, order=1).reshape(shape)

In [6]:
def norm_input(x):
    return (x - mean_px) / std_px

In [7]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
history = AccuracyHistory()


In [8]:

read_train_filename = './all/train_images2.npy'

train_label_filename = './all/train_labels.csv'
train_data_filename = './train_data.npy'
class_name_list =["sink","pear","moustache","nose","skateboard","penguin","peanut","skull","panda","paintbrush","nail","apple","rifle","mug","sailboat","pineapple","spoon","rabbit","shovel","rollerskates","screwdriver","scorpion","rhinoceros","pool","octagon","pillow","parrot","squiggle","mouth","empty","pencil"]
num_classes = 31
size = crop_all_images('./all/train_images.npy','./all/train_images2.npy')#('comp-551-kaggle-master/all/train_images.npy','comp-551-kaggle-master/all/train_images2.npy')
resize_all('./all/train_images2.npy','./all/train_images2.npy',size,image_colmn)
#code example taking the images in test_images.npy, cropping them, then rescaling them
size = crop_all_images('./all/test_images.npy','./all/test_images2.npy')
resize_all('./all/test_images2.npy','./all/test_images2.npy',size,image_colmn)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:50<00:00, 196.70it/s]


10000


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [01:01<00:00, 162.78it/s]


10000


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [01:00<00:00, 165.13it/s]


In [9]:
#checking processed data is exist.
read_train_filename = './all/train_images2.npy'
if (os.path.isfile(train_data_filename)==False):
    train_data = create_train_data()
        
else:
    print("file already exist")
    train_data = np.load(train_data_filename,encoding='latin1')

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 73452.58it/s]


In [10]:
read_test_filename = './all/test_images2.npy'

test_data_filename = './test_data.npy'

if (os.path.isfile(test_data_filename)==False):
    images = np.load(read_test_filename,encoding='latin1')
    predict_data=[]
    for i in tqdm(range(len(images))):
            predict_data.append([np.array((images[i][1])).reshape(1,image_colmn*image_row)])
    np.save('test_data.npy',predict_data)
    
else:
    print("file already exist")
    predict_data = np.load(test_data_filename,encoding='latin1')
    


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 64273.92it/s]


In [11]:
print(len(predict_data))

10000


In [12]:
aug_epochs = 50
test_size = 0.05
#seed 
random_seed = 1
learning_rate=0.001
version =1
batch_size=128
epochs=40


y_raw = [i[1] for i in train_data]
#print(y_raw)
#for y_e in y_raw:
#    y.append(convert_y_to_vector(y_e))
train_y=np.asarray(y_raw,dtype=float)

#print(y)

train_y = train_y.reshape((-1,1))




train_x = np.array([i[0] for i in train_data])
train_x = train_x.reshape((-1,image_colmn,image_row))
input_shape = (image_colmn, image_row, 1)
predict_x = np.array([i[0] for i in predict_data])

predict_x =predict_x.reshape((-1,image_colmn,image_row,1))

    ###############################################################################
    # Perspective skew
p = Augmentor.Pipeline()
p.skew(probability=1, magnitude=0.2)
p.status()
#print(x_train.shape)
g = p.keras_generator_from_array(train_x, train_y, batch_size=train_x.shape[0])
skew_1, skew_y_1 = next(g)
#print(skew_1.shape)
skew_2, skew_y_2 = next(g)
skew_1 = skew_1.reshape(train_x.shape)
skew_2 = skew_2.reshape(train_x.shape)
    ###############################################################################

Operations: 1
	0: Skew (skew_type=RANDOM magnitude=0.2 probability=1 )
Images: 0
Classes: 0

You can remove operations using the appropriate index and the remove_operation(index) function.


In [13]:
distort_1 = np.zeros(shape=train_x.shape, dtype='float32')
distort_2 = np.zeros(shape=train_x.shape, dtype='float32')
distort_3 = np.zeros(shape=train_x.shape, dtype='float32')

for i in range(train_x.shape[0]):
        #print(i)
        distort_1[i] = elastic_transform(image=train_x[i], alpha=36, sigma=10)
        distort_2[i] = elastic_transform(image=train_x[i], alpha=36, sigma=8)
        distort_3[i] = elastic_transform(image=train_x[i], alpha=36, sigma=6)
train_x =   np.concatenate((train_x, distort_1, distort_2, distort_3, skew_1, skew_2), axis=0)
train_y = np.concatenate((train_y, train_y, train_y, train_y, skew_y_1, skew_y_2), axis=0)


In [14]:
train_x = train_x.reshape((-1,image_colmn,image_row,1))
train_y = train_y.reshape((train_y.shape[0],))
train_x = train_x.astype('float32')
predict_x = predict_x.astype('float32')
train_x /= 255
predict_x /= 255

In [15]:
print(predict_x.shape)

(10000, 40, 40, 1)


In [16]:
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=test_size, random_state=111)
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)


In [17]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
mean_px = x_train.mean().astype(np.float32)
std_px = x_train.std().astype(np.float32)


In [18]:
#https://keras.io/getting-started/sequential-model-guide/
model = Sequential([
        Lambda(norm_input, input_shape=input_shape),

        Conv2D(32, strides=(1, 1),kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape),
        Conv2D(32, strides=(1, 1),kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2),strides=(1, 1)),
        Dropout(0.25),

        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        # Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        # Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        # Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        # BatchNormalization(),
        # MaxPooling2D(pool_size=(2, 2)),
        # Dropout(0.25),

        Flatten(),

        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        # Dense(512, activation='relu'),
        # BatchNormalization(),
        # Dropout(0.5),
        # Dense(512, activation='relu'),
        # BatchNormalization(),
        # Dropout(0.5),
        Dense(31, activation='softmax')
    ])

In [20]:
model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=learning_rate),
                  metrics=['accuracy'])   #https://keras.io/models/sequential/
original = sys.stdout
sys.stdout = open('history_' + str(version) + '.txt', 'w')

model.summary()#https://keras.io/models/about-keras-models/#about-keras-models
#https://keras.io/models/sequential/
history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test),
                        class_weight=class_weights,
                        callbacks =[history])

In [21]:
gen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, shear_range=0.3,
                             height_shift_range=0.1, zoom_range=0.08)  # changed from 8, 0.08, 0.3, 0.08
#https://keras.io/preprocessing/image/#imagedatagenerator-class


batches = gen.flow(x_train, y_train, batch_size=batch_size)

val_batches = gen.flow(x_test, y_test, batch_size=batch_size)

model.fit_generator(batches, steps_per_epoch=x_train.shape[0] // batch_size, epochs=aug_epochs,
                        validation_data=val_batches, validation_steps=x_test.shape[0] // batch_size,
                        use_multiprocessing=False, class_weight=class_weights)

In [22]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [23]:
model_json = model.to_json()
json_name = "model_" + str(version) + ".json"
h5_name = "model_" + str(version) + ".h5"
with open(json_name, "w") as json_file:
    json_file.write(model_json)
model.save_weights(h5_name)
print("Saved model to disk")

In [26]:
predict_result = model.predict_classes(predict_x, batch_size=batch_size, verbose=0)
with open ('submission.csv','w',) as csvfile: 
    csvfile.write('Id')
    csvfile.write(",")
    csvfile.write('Category')
    csvfile.write('\n')
    for i in range(len(predict_result)):
            csvfile.write('%d'%i)
            csvfile.write(",")
            csvfile.write(class_name_list[predict_result[i]])
            csvfile.write('\n')

In [27]:
plt.plot(range(1,11), history.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

AttributeError: 'History' object has no attribute 'acc'